# Introduction
Use PyTorch to develop a binary classifier for the Diabetes dataset

In [14]:
# Import Standard Libraries
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from sklearn.preprocessing import StandardScaler

# Read Data

In [2]:
# Read data
data = pd.read_csv('./../../data/diabetes.csv')

In [3]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
# Define Features & Label as arrays
x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Data Preprocessing

In [5]:
# Data Normalisation
standard_scaler = StandardScaler()

# Fit and transform X
x = standard_scaler.fit_transform(x)

In [6]:
# Convert to PyTorch tensor
x = torch.tensor(x)
y = torch.tensor(y).unsqueeze(1) # Transform the shape from (n) to (n, 1)

In [15]:
# Define the Dataset for manipulating the data in PyTorch
class Dataset(Dataset):
    
    def __init__(self, x, y):
        
        self.x = x
        self.y = y
        
    def __getitem(self, index):
        
        return self.x[index], self.y[index]
    
    def __len__(self):
        
        return len(self.x)

In [16]:
# Create the Dataset object
dataset = Dataset(x, y)

In [18]:
# Define the DataLoader for loading the data into the Neural Network in batches
train_data_loader = DataLoader(dataset,
                               batch_size=32,
                               shuffle=True)